# Trading Strategy Optimization

This notebook performs parameter optimization for a trading strategy using historical market data. The optimization process uses Optuna to find the best combination of parameters that maximize the strategy's performance.

In [ ]:
import sys
sys.path.append('..')

import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import re
from datetime import datetime, time, timedelta

PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname('__file__'), '..'))
RESULTS_DIR = os.path.join(PROJECT_ROOT, 'results')
DATA_CACHE_DIR = os.path.join(PROJECT_ROOT, 'data_cache')
LOGS_DIR = os.path.join(PROJECT_ROOT, 'logs')
CONFIG_DIR = os.path.join(PROJECT_ROOT, 'config')

os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(DATA_CACHE_DIR, exist_ok=True)
os.makedirs(LOGS_DIR, exist_ok=True)

from src.data.data_loader import DataLoader
from src.data.data_processor import DataProcessor
from src.indicators.bollinger_bands import add_bollinger_bands
from src.indicators.rsi import add_rsi
from src.indicators.atr import add_atr
from src.strategy.signal_generator import SignalGenerator
from src.strategy.risk_manager import RiskManager
from src.backtest.backtest_engine import BacktestEngine
from src.backtest.performance import PerformanceMetrics
from src.optimization.strategy_optimizer import StrategyOptimizer
from src.visualization.optimization import plot_optimization_progress, plot_parameter_importance, plot_timeframe_analysis, plot_parameter_space
from src.visualization.comparison import plot_parameter_comparison, plot_equity_curves_comparison

sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (14, 8)

## 1. Configuration Setup

Load the configuration settings for the optimization process, including timeframes to test and parameter ranges.

In [ ]:
from src.optimization.config_loader import ConfigLoader

config_loader = ConfigLoader()
config = config_loader.get_config()

optimization_config = config.get("optimization", {})
timeframes = optimization_config.get("timeframes", ["5min", "15min", "30min", "1h", "4h"])
param_ranges = optimization_config.get("parameter_ranges", {})

print("\nOptimization Configuration:")
print(f"Timeframes to test: {timeframes}")
print("\nParameter ranges:")
for param, range_dict in param_ranges.items():
    print(f"- {param}: {range_dict}")

## 2. Define Training and Testing Periods

Set the date ranges for the training and validation periods, as well as the number of optimization trials.

In [ ]:
train_start_date = '2024-01-01'
train_end_date = '2024-06-01'
test_start_date = '2024-06-01' 
test_end_date = '2025-01-01'

N_TRIALS = 100

print(f"Training period: {train_start_date} to {train_end_date}")
print(f"Validation period: {test_start_date} to {test_end_date}")
print(f"Number of optimization trials: {N_TRIALS}")

## 3. Load Market Data

Load historical market data for both training and validation periods.

In [ ]:
loader = DataLoader(cache_dir=DATA_CACHE_DIR)

print("Loading training data...")
train_data = loader.get_active_contract_data(train_start_date, train_end_date)
print(f"Loaded {len(train_data)} tick data points for training period")

print("\nLoading validation data...")
test_data = loader.get_active_contract_data(test_start_date, test_end_date)
print(f"Loaded {len(test_data)} tick data points for validation period")

if train_data.empty or test_data.empty:
    raise ValueError("Failed to load market data. Please check database connection and date range.")

## 4. Initialize Strategy Optimizer

Create an instance of the StrategyOptimizer with the loaded data.

In [ ]:
optimizer = StrategyOptimizer(
    train_data=train_data,
    test_data=test_data,
    initial_capital=100000,
    n_trials=N_TRIALS,
    timeframes=timeframes,
)

## 5. Run Optimization Process

Execute the optimization process to find the best parameter combination.

In [ ]:
print(f"Starting optimization with {N_TRIALS} trials...")
best_params = optimizer.optimize()

print("\nOptimization complete!")
print(f"Best parameters: {best_params}")

## 6. Analyze Optimization Results

Load and analyze the results of the optimization process.

In [ ]:
from src.optimization.optimization_analyzer import OptimizationAnalyzer
analyzer = OptimizationAnalyzer(log_file="logs/optimization_results.log")
opt_df = analyzer.trials_df
print(f"Found {len(opt_df)} optimization trials")
opt_df.head()

## 7. Visualize Optimization Results

Generate plots to visualize the optimization process and results.

In [ ]:
fig_progress = analyzer.plot_optimization_progress(save_path=os.path.join(RESULTS_DIR, 'optimization_progress.png'))
plt.show()

In [ ]:
fig_importance = analyzer.plot_parameter_importance(save_path=os.path.join(RESULTS_DIR, 'parameter_importance.png'))
plt.show()

In [ ]:
fig_timeframe, timeframe_analysis = analyzer.plot_timeframe_analysis(save_dir=RESULTS_DIR)
plt.show()
print("Performance by Timeframe:")
display(timeframe_analysis)

## 8. Extract Top Parameter Combinations

Identify the best parameter combinations from the optimization trials.

In [ ]:
top_params = analyzer.get_top_parameters(n=5)
print("Top 5 parameter combinations:")
display(top_params)
top_params.to_csv(os.path.join(RESULTS_DIR, 'top_parameters.csv'), index=False)

## 9. Validate Best Parameters

Test the best parameters on the validation dataset and generate performance metrics.

In [ ]:
summary = analyzer.generate_summary_report(save_dir=RESULTS_DIR)
best_params = summary['best_parameters']

print("Best parameters from optimization:")
for param, value in best_params.items():
    print(f"- {param}: {value}")

validation_results, metrics = optimizer.validate_best_parameters(best_params)

print("\nOut-of-Sample Validation Results:")
if metrics is not None:
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}" if isinstance(value, (int, float)) else f"{metric}: {value}")
else:
    print("Validation failed or no trades were executed")

if validation_results and 'portfolio_history' in validation_results and len(validation_results['portfolio_history']) > 1:
    from src.visualization.backtest import plot_equity_curve
    fig = plot_equity_curve(
        validation_results['portfolio_history'],
        initial_capital=100000,
        save_path=os.path.join(RESULTS_DIR, 'validation_equity_curve.png')
    )
    plt.show()
else:
    print("No equity curve available - insufficient data")

## 10. Compare Default vs. Optimized Parameters

Compare the performance of the default parameters against the optimized parameters.

In [ ]:
from src.pipeline import TradingPipeline


default_params = config['parameters'].copy()
print("Running backtest with default parameters...")

pipeline_default = TradingPipeline(config)
default_results, default_signals_df = pipeline_default.run_backtest(test_start_date, test_end_date, default_params['default_timeframe'])

if default_results and 'trades' in default_results and not default_results['trades'].empty:
    default_performance = PerformanceMetrics(
        default_results['trades'],
        default_results['portfolio_history']
    )
    default_metrics = default_performance.generate_report()
    
    print("Performance with Default Parameters:")
    for metric, value in default_metrics.items():
        print(f"{metric}: {value:.4f}" if isinstance(value, (int, float)) else f"{metric}: {value}")
else:
    default_metrics = None
    print("No trades were executed with default parameters")

if default_metrics and metrics:
    fig_comparison, comparison_df = plot_parameter_comparison(
        default_metrics, 
        metrics, 
        save_path='results/metrics_comparison.png'
    )
    plt.show()
    
    print("Comparison of Default vs Optimized Parameters:")
    display(comparison_df)
    comparison_df.to_csv(os.path.join(RESULTS_DIR, 'parameter_comparison.csv'))
    
    fig_curves = plot_equity_curves_comparison(
        default_results['portfolio_history'],
        validation_results['portfolio_history'],
        save_path=os.path.join(RESULTS_DIR, 'default_vs_optimized.png')
    )
    plt.show()
else:
    print("Cannot compare metrics - insufficient data")

## 11. Save Optimized Parameters

Save the optimized parameters to configuration files for future use.

In [ ]:
best_trial = analyzer.get_top_parameters(n=1).iloc[0].to_dict()

optimized_config = config.copy()

optimized_config['parameters']['bb_window'] = best_params['bb_window']
optimized_config['parameters']['bb_std'] = best_params['bb_std']
optimized_config['parameters']['rsi_period'] = best_params['rsi_period']
optimized_config['parameters']['rsi_lower'] = best_params['rsi_lower']
optimized_config['parameters']['rsi_upper'] = best_params['rsi_upper']
optimized_config['parameters']['atr_period'] = best_params['atr_period']
optimized_config['parameters']['take_profit_mult'] = best_params['take_profit_mult']
optimized_config['parameters']['stop_loss_mult'] = best_params['stop_loss_mult']

if 'timeframe' in best_params:
    optimized_config['parameters']['default_timeframe'] = best_params['timeframe']
else:
    optimized_config['parameters']['default_timeframe'] = best_trial.get('timeframe', config['parameters']['default_timeframe'])

optimized_config['optimization_results'] = {
    'date': datetime.now().strftime('%Y-%m-%d %H:%M'),
    'train_period': f'{train_start_date} to {train_end_date}',
    'test_period': f'{test_start_date} to {test_end_date}',
    'trials': N_TRIALS,
    'best_score': float(best_trial['score']) if 'score' in best_trial else float(best_params.get('score', 0))
}

if metrics:
    optimized_config['performance'] = {
        metric: float(value) if isinstance(value, (int, float, np.number)) else value 
        for metric, value in metrics.items()
        if metric in ['Win Rate', 'Profit Factor', 'Sharpe Ratio', 'Maximum Drawdown', 'Total Return', 'Total Trades']
    }

output_path = os.path.join(CONFIG_DIR, 'optimized_parameters.json')
with open(output_path, 'w') as f:
    json.dump(optimized_config, f, indent=4)

print(f'Optimized parameters saved to {output_path}')

optimized_params = {
    'parameters': best_params,
    'optimization_info': {
        'date': datetime.now().strftime('%Y-%m-%d %H:%M'),
        'train_period': f'{train_start_date} to {train_end_date}',
        'test_period': f'{test_start_date} to {test_end_date}',
        'best_score': float(best_trial['score']) if 'score' in best_trial else float(best_params.get('score', 0))
    },
    'performance': optimized_config.get('performance', {})
}

with open(os.path.join(RESULTS_DIR, 'optimized_parameters.json'), 'w') as f:
    json.dump(optimized_params, f, indent=4)
    
print(f'Optimized parameters also saved to {os.path.join(RESULTS_DIR, "optimized_parameters.json")}')